### 1. Instalação das bibliotecas necessárias

In [1]:
%pip install requests pandas plyer

Note: you may need to restart the kernel to use updated packages.


### 2. Importação das bibliotecas

In [2]:
import requests
import pandas as pd
import sqlite3
from plyer import notification
from datetime import datetime

### 3. Def de criação do alerta

In [3]:
def alerta(nivel, base, etapa):
  if nivel == 1:
    alerta = 'Baixo'
  elif nivel == 2:
    alerta = 'Médio'
  else:
    alerta = 'Alto'

  title = f"Alerta {alerta}"
  message = f"Falha no carregamento da base {base} na etapa {etapa} - {datetime.now()}"
  notification.notify(
    title=title,
    message=message,
    app_name="etl_pipeline",
    timeout=10
  )

### 4. Def para extrair dados da API

In [4]:
def extrair_dados_da_api(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        alerta(3, "PokeAPI", "Extração")
        raise Exception(f"Falha ao buscar dados da API. Código de status: {response.status_code}")

### 5. Def para extrair dados detalhados dos Pokemóns

In [5]:
def extrair_detalhes_pokemon(url):
    data = extrair_dados_da_api(url)
    habilidades = [habilidade['ability']['name'] for habilidade in data['abilities']]
    return habilidades

### 6. Def para extrair 3 tabelas da API e relacionar dados

In [6]:
def extrair_tabelas(base_url):
    pokemons_data = extrair_dados_da_api(f"{base_url}/pokemon?limit=10")
    habilidades_list = []
    
    for pokemon in pokemons_data['results']:
        habilidades = extrair_detalhes_pokemon(pokemon['url'])
        habilidades_list.append(habilidades)
    
    linguagens_data = extrair_dados_da_api(f"{base_url}/language")
    linguagens_list = linguagens_data['results']
    
    df_pokemons = pd.DataFrame(pokemons_data['results'])
    df_habilidades = pd.DataFrame(habilidades_list, columns=[f'habilidade_{i+1}' for i in range(len(habilidades_list[0]))])
    df_linguagens = pd.DataFrame(linguagens_list)
    
    return df_pokemons, df_habilidades, df_linguagens

### 7. Def para carregar os dados no banco de dados

In [7]:
def carregar_dados_para_db(df, db_name="../pokeapi_data.db", table_name="tabela"):
    conn = sqlite3.connect(db_name)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()
    print(f"Dados carregados para {db_name} na tabela {table_name}")

### 8. Execução da extração, transformação e carga dos dados

In [8]:
url = "https://pokeapi.co/api/v2"

In [9]:
# Extração dos dados
df_pokemons, df_habilidades, df_linguagens = extrair_tabelas(url)

In [10]:
# Carregar dados no banco de dados
carregar_dados_para_db(df_pokemons, db_name="../pokeapi_data.db", table_name="pokemons")
carregar_dados_para_db(df_habilidades, db_name="../pokeapi_data.db", table_name="habilidades")
carregar_dados_para_db(df_linguagens, db_name="../pokeapi_data.db", table_name="linguagens")

Dados carregados para ../pokeapi_data.db na tabela pokemons
Dados carregados para ../pokeapi_data.db na tabela habilidades
Dados carregados para ../pokeapi_data.db na tabela linguagens


#### 9. Visualização dos dados usando queries SQL

In [11]:
def executar_query(query, db_name="../pokeapi_data.db"):
    conn = sqlite3.connect(db_name)
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

In [12]:
# Dados da tabela Pokemóns
query_pokemons = "SELECT * FROM pokemons"
df_pokemons_query = executar_query(query_pokemons)
print(df_pokemons_query.head())

         name                                   url
0   bulbasaur  https://pokeapi.co/api/v2/pokemon/1/
1     ivysaur  https://pokeapi.co/api/v2/pokemon/2/
2    venusaur  https://pokeapi.co/api/v2/pokemon/3/
3  charmander  https://pokeapi.co/api/v2/pokemon/4/
4  charmeleon  https://pokeapi.co/api/v2/pokemon/5/


In [13]:
# Dados da tabela Habilidades
query_habilidades = "SELECT * FROM habilidades"
df_habilidades_query = executar_query(query_habilidades)
print(df_habilidades_query.head())

  habilidade_1 habilidade_2
0     overgrow  chlorophyll
1     overgrow  chlorophyll
2     overgrow  chlorophyll
3        blaze  solar-power
4        blaze  solar-power


In [14]:
# Dados da tabela Linguagens
query_linguagens = "SELECT * FROM linguagens"
df_linguagens_query = executar_query(query_linguagens)
print(df_linguagens_query.head())

      name                                    url
0  ja-Hrkt  https://pokeapi.co/api/v2/language/1/
1  roomaji  https://pokeapi.co/api/v2/language/2/
2       ko  https://pokeapi.co/api/v2/language/3/
3  zh-Hant  https://pokeapi.co/api/v2/language/4/
4       fr  https://pokeapi.co/api/v2/language/5/
